In [1]:
import numpy as np
import pandas as pd
import re
import yfinance as yf

Loading data

In [2]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [3]:
df = df.loc[df['TYPE'] == 'MERGED']

Extracting Time and Data

In [4]:
df["Time"] = df.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df["DATE"] = df.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

Cleaning and organizing

In [6]:
df['DATE'] = pd.to_datetime(
                          df['DATE'],
                          format='%Y-%m-%d')

In [7]:
df = df.sort_values(by="DATE")

In [8]:
df.columns = ['DATE', 'ID', 'STOCK',"SECTOR","TYPE","SCORE","TIME"]

In [9]:
df['SCORE']=df['SCORE'].replace(0, np.nan)

Adding Returns